In [ ]:
!pip install --quiet arcgis

In [2]:
import os;
from arcgis.gis import GIS;
from arcgis.mapping import WebMap;
from arcgis.features import SpatialDataFrame
import pandas as pd;
import arcpy;


In [3]:
fp_path  = r"F:\SitingStagingCache";
fp_fgdb  = r"footprint.gdb";
fp_full  = fp_path + os.sep + fp_fgdb;
fp_layer = "EsriSSURGO";
fp_trg   = fp_path + os.sep + "EsriSSURGO" + os.sep + "trg";

resources = [f.name for f in os.scandir(fp_trg) if f.is_dir()];

len(resources)

2204

In [4]:
if not os.path.exists(fp_full):
    arcpy.CreateFileGDB_management(fp_path,fp_fgdb);
    
if arcpy.Exists(fp_full + os.sep + fp_layer):
    arcpy.Delete_management(fp_full + os.sep + fp_layer);
    
arcpy.CreateFeatureclass_management(
     out_path          = fp_full
    ,out_name          = fp_layer
    ,geometry_type     = "POLYGON"
    ,has_m             = "DISABLED"
    ,has_z             = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269)
    ,config_keyword    = None
);

arcpy.env.workspace = fp_full;
arcpy.management.AddFields(
     fp_layer
    ,[
         ['resource_name' ,'TEXT' ,'Resource_Name'  ,255, None,'']
        ,['resource_path' ,'TEXT' ,'Resource_Path'  ,255, None,'']
     ]
);

In [ ]:
cursor = arcpy.da.InsertCursor(
     fp_full + os.sep + fp_layer
    ,["resource_name","resource_path","SHAPE@"]
);

for item in resources:
    basin = fp_trg + os.sep + item + os.sep + "Subbasin";
    
    poly = arcpy.da.SearchCursor(
         in_table    = basin
        ,field_names = ["SHAPE@"]
    );
    
    for row in poly:
        shp = row[0];
    
    cursor.insertRow([item,basin,shp]);

if cursor:
    del cursor;
    
'footprints generated'